In [4]:
class full_game:
    def __init__(self, no_players, Human, shuffle = False):
        self.player_names = ['Martha', 'Lorenzo', 'Maria', 'Manuel', 'Ana', 'Liam'] 
        self.Players = []
        self.palafico = False
        self.winner=None
        if Human == True:
            no_players = int(input("How many other players would you like to play with?: ")) #Maximum is 5
            self.human = str(input("What is your name?: "))
            self.Players.append(self.human)
        for i in range(no_players):
            self.Players.append(self.player_names[i])
        if shuffle == True:
            rnd.shuffle(self.Players) #Shuffle!!! Only in __innit__ as seating must be fixed
        self.diceState = {}
        for i in self.Players:
            self.diceState[i] = 5 #Assigning every player 5 dice at the start
        self.player = rnd.choice(self.Players)

    # this function updates the dicestate after a round, if a player has 0 dice, it eliminates 
    # them and updates the diceState and Players list!
    def update_dice_players(self, player, dice):
        if dice == 1:
            if self.diceState[player] < 5:
                self.diceState[player] += dice #Can be +1 if they exactly, or -1 if they loose a round
        else:
            self.diceState[player] += dice
        if self.diceState[player] == 0:
            indx = self.Players.index(player)
            if indx == len(self.Players)-1:
                self.player = self.Players[0]
            else:
                self.player = self.Players[indx+1]
            self.Players.remove(player) #Remove player from list of players
            del self.diceState[player] #Remove player from diceState list
        elif self.diceState[player] ==1:
            self.palafico = True
            self.player = player
        else:
            self.player = player

In [2]:
class rounds:
    def __init__(self, diceState, Players, current_player, human, shuffle=False, palafico= False):
        self.human = human
        self.history = [] #Stores all the bids in this round
        self.last = None #Last bid - could be combined with above?
        self.loser = []
        self.player = current_player #Who is the current player
        self.dice_state = diceState #How many dice each player has
        self.Players = Players #Players still in the game
        self.count = {} #Count how many x's of y there are on the table i.e 3 2's.      
        self.rolls = {}
        self.reason = []
        self.phase = []
        self.winner = []
        self.palafico= palafico
        if shuffle == True:
            rnd.shuffle(self.Players) 
        for i in self.Players: #Assign random rolls to each player
            self.rolls[i] = [rnd.randint(1, 6) for i in range(self.dice_state[i])]
        self.total_dice = 0 #Update total number of dice
        for i in self.Players:
            self.total_dice += self.dice_state[i]
        if shuffle == True:
            self.Players 
    
    def next_player(self): #This make sense?
        self.last_player = self.player
        for i in range(len(self.Players)):
            if self.Players[i] == self.player:
                if i != (len(self.Players)-1):
                    self.player = self.Players[i+1]
                    break
                else:
                    self.player = self.Players[0]
                    break
    
    def check_if_terminal(self): #This make sense?
        if self.last == 'Dudo' or self.last == 'Calza':
            return True
        return False    
    
    def call(self, Dudo = True, Calza = False): #This creates a loser of the round
        Aces = Twos = Threes = Fours = Fives = Sixes = 0
        dice_val_names = ['Aces', 'Twos', 'Threes', 'Fours', 'Fives', 'Sixes']
        dice_vals = (1,2,3,4,5,6)
        for y in self.rolls: #For each player
            Aces += self.rolls[y].count(1)
            Twos += self.rolls[y].count(2)
            Threes += self.rolls[y].count(3)
            Fours += self.rolls[y].count(4)
            Fives += self.rolls[y].count(5)
            Sixes += self.rolls[y].count(6)
        dice_count = [Aces, Twos, Threes, Fours, Fives, Sixes]
        for x in range(len(dice_vals)): 
            self.count[dice_val_names[x]] = dice_count[x]
            if self.history[-2][1] == dice_vals[x]:
                if x == 0: #Aces!!
                    if self.history[-2][0] > self.count[dice_val_names[x]]:
                        if Dudo == True:
                            self.loser = self.last_player
                            self.reason = 'Bid too high'
                        elif Calza == True:
                            self.loser = self.player
                    else:
                        if Calza == True:
                            if self.history[-2][0] == self.count[dice_val_names[x]]:
                                self.winner = self.player
                            else:
                                self.loser = self.player
                        else:
                            self.loser = self.player
                            self.reason = 'Dudo too soon'
                else: #Include the Aces
                    if self.history[-2][0] > (self.count[dice_val_names[x]] + self.count['Aces']):
                        if Dudo == True:
                            self.loser = self.last_player
                            self.reason = 'Bid too high'
                        elif Calza == True:
                            self.loser = self.player
                    else:
                        if Calza == True:
                            if  self.history[-2][0] == (self.count[dice_val_names[x]] + self.count['Aces']):
                                self.winner = self.player
                            else:
                                self.loser = self.player
                        else: 
                            self.loser = self.player
                            self.reason = 'Dudo too soon'
    
        if self.winner:
            return [self.winner, +1]
        else:
            return [self.loser, -1]   
        

        